In [1]:
import pandas as pd
from google.cloud import tpu_v2
from datetime import datetime, timedelta
from dateutil import parser

In [2]:
project_id = 'picodo-455019'

In [3]:
# list all TPUs across all zones
client = tpu_v2.TpuClient()
parent = f'projects/{project_id}/locations/-'
nodes = client.list_nodes(parent=parent)

# create table
rows = []
for node in nodes:
    time_created = node.create_time
    time_now = datetime.now(node.create_time.tzinfo)
    time_last = time_now if not 'maintenance' in node.health_description else parser.parse(node.health_description, fuzzy=True)
    row = {
        'id': node.name.split('/')[-1],
        'zone': node.name.split('/')[3],
        'type': node.accelerator_type,
        'spot': node.scheduling_config.spot,
        'state': node.state.name,
        'created': (time_now - time_created).total_seconds(),
        'uptime': (time_last - time_created).total_seconds(),
        #'health': node.health.name,
        #'health desc.': node.health_description,
    }
    rows += [row]
df = pd.DataFrame(rows)

# sort table
df.sort_values(by='created', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)

# format table
def highlight_rows(row):
    color_map = {
        'READY': 'background-color: #a2de92',
        'CREATING': 'background-color: #fffc7f',
        'DELETING': 'background-color: #ff963b',
        'PREEMPTED': 'background-color: #ffb8b5',
    }
    return [color_map.get(row['state'], '') for _ in row]
df.style.apply(highlight_rows, axis=1).format({
    'created': lambda x: timedelta(seconds=round(x)),
    'uptime': lambda x: timedelta(seconds=round(x)),
}).set_table_styles([
    {'selector': 'th, td', 'props': [('border', '1px solid #000000')]}
])

,id,zone,type,spot,state,created,uptime
0,my-tpu-node,us-central2-b,v4-64,False,READY,"8 days, 3:45:23","8 days, 3:45:23"
1,my_spot_tpu,us-east1-d,v6e-4,True,PREEMPTED,"2 days, 18:10:56",0:13:12
2,spot-tpu-62c019,us-east1-d,v6e-4,True,PREEMPTED,"2 days, 16:29:19",6:26:44
3,tpu-v6e-8-spot-1,europe-west4-a,v6e-8,True,PREEMPTED,"2 days, 3:21:54",5:46:16
4,nanny3,europe-west4-a,v6e-1,True,READY,1:14:22,1:14:22
5,nanny4,europe-west4-a,v6e-1,True,READY,1:10:31,1:10:31
6,nanny5,europe-west4-a,v6e-1,True,READY,1:07:31,1:07:31
